# Interior Reimaginer: 3D Reconstruction in Google Colab

This notebook demonstrates how to use the 3D reconstruction features of Interior Reimaginer in Google Colab.

## Setup

First, we'll clone the repository and install the required dependencies:

In [ ]:
# Clone the repository
!git clone https://github.com/your-username/CV_Project.git
%cd CV_Project

In [ ]:
# Install dependencies
!pip install -r interior_reimaginer/requirements.txt
!pip install open3d matplotlib

## Import Required Modules

Now let's import the necessary modules and initialize our models:

In [ ]:
from interior_reimaginer.models.interior_reimaginer import InteriorReimaginer
from interior_reimaginer.models.reconstruction_3d import DepthReconstructor
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import time

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Initialize the models
print("Initializing models...")
reimaginer = InteriorReimaginer(device=device)
depth_reconstructor = DepthReconstructor()
print("Models initialized successfully!")

## Upload or Load an Image

You can either upload an image or use a sample image:

In [ ]:
# Option 1: Upload an image
from google.colab import files
uploaded = files.upload()  # This will prompt to upload an image

import io
image_name = list(uploaded.keys())[0]
image = Image.open(io.BytesIO(uploaded[image_name]))

# Display the uploaded image
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.title("Uploaded Image")
plt.axis('off')
plt.show()

## Process the Image

Now we'll process the image to get the depth map and other features:

In [ ]:
print("Processing the image...")
processed = reimaginer.image_processor.process_image(image)
print("Image processing complete!")

# Display the depth map
if processed.depth_map is not None:
    plt.figure(figsize=(12, 8))
    plt.imshow(processed.depth_map, cmap='inferno')
    plt.title("Generated Depth Map")
    plt.colorbar(label="Depth")
    plt.axis('off')
    plt.show()
else:
    print("Depth map generation failed.")

## 3D Reconstruction - Point Cloud

Create a 3D point cloud from the depth map:

In [ ]:
# Set downsampling factor (adjust for quality vs. performance)
downsample_factor = 2  # Higher values = faster but less detailed

print(f"Creating point cloud with downsample factor {downsample_factor}...")
pcd = depth_reconstructor.depth_to_pointcloud(
    depth_map=processed.depth_map,
    image=image,
    downsample_factor=downsample_factor
)
print(f"Point cloud created with {len(pcd.points)} points")

# Visualize the point cloud
print("Rendering point cloud preview...")
preview_img = depth_reconstructor.render_pointcloud_image(pcd)
plt.figure(figsize=(12, 8))
plt.imshow(preview_img)
plt.title("Point Cloud Preview")
plt.axis('off')
plt.show()

# Interactive visualization (may not work in all Colab environments)
try:
    print("Opening interactive point cloud viewer (close the window to continue)...")
    depth_reconstructor.visualize_pointcloud(pcd)
except Exception as e:
    print(f"Interactive visualization failed: {e}")
    print("This is expected in some Colab environments without display access.")

## 3D Reconstruction - Mesh

Convert the point cloud to a mesh:

In [ ]:
print("Creating mesh from point cloud...")
try:
    mesh = depth_reconstructor.pointcloud_to_mesh(pcd)
    print(f"Mesh created with {len(mesh.triangles)} triangles")
    
    # Visualize the mesh
    print("Rendering mesh preview...")
    preview_img = depth_reconstructor.render_mesh_image(mesh)
    plt.figure(figsize=(12, 8))
    plt.imshow(preview_img)
    plt.title("Mesh Preview")
    plt.axis('off')
    plt.show()
    
    # Interactive visualization (may not work in all Colab environments)
    try:
        print("Opening interactive mesh viewer (close the window to continue)...")
        depth_reconstructor.visualize_mesh(mesh)
    except Exception as e:
        print(f"Interactive visualization failed: {e}")
        print("This is expected in some Colab environments without display access.")
except Exception as e:
    print(f"Mesh creation failed: {e}")

## Save the 3D Models

Save the point cloud and mesh to files:

In [ ]:
# Generate a unique timestamp for the filenames
timestamp = int(time.time())

# Save point cloud
pcd_path = depth_reconstructor.save_pointcloud(pcd, f"interior_pointcloud_{timestamp}")
print(f"Point cloud saved to: {pcd_path}")

# Save mesh if it was created
try:
    if 'mesh' in locals():
        mesh_path = depth_reconstructor.save_mesh(mesh, f"interior_mesh_{timestamp}")
        print(f"Mesh saved to: {mesh_path}")
except Exception as e:
    print(f"Error saving mesh: {e}")

## Download the 3D Models

Download the generated 3D models to your local machine:

In [ ]:
from google.colab import files

# Download the point cloud file
files.download(pcd_path)

# Download the mesh file if it exists
try:
    if 'mesh_path' in locals():
        files.download(mesh_path)
except Exception as e:
    print(f"Error downloading mesh: {e}")

## Conclusion

You have now successfully:
1. Processed an interior image to extract its depth map
2. Created a 3D point cloud from the depth map
3. Generated a 3D mesh from the point cloud
4. Saved and downloaded the 3D models

These 3D models can be imported into other 3D software like Blender, Unity, or other 3D viewers for further use.